In [ ]:
import pandas as pd
import ollama
from ddgs import DDGS
import csv
import time

MODEL_NAME = "llama3.1"
INPUT_FILE = "overbase_list.csv"
MISSING_DATA_FILE = "missing_data_rows.csv"
OUTPUT_FILE = "processed_leads_sample.csv"
BATCH_SIZE = 10

In [ ]:
def clean_and_load_data(filepath):
    valid_rows = []
    missing_data_rows = []
    
    # Define all variations of "missing" found in your file
    INVALID_VALUES = {'', '-', '—', '–', 'n/a', 'nan', 'none'}

    try:
        with open(filepath, 'r', encoding='utf-8', errors='replace') as f:
            reader = csv.reader(f)
            header = next(reader, None) # Skip header
            
            for row in reader:
                if not row: continue
                
                name, title, company = None, None, None
                
                # Logic for column variations
                if len(row) == 4:
                    name, title, company = row[0], row[1], row[2]
                elif len(row) >= 5:
                    name, title, company = row[0], row[1], row[3] # 4th column is company
                else:
                    missing_data_rows.append(row)
                    continue

                # 1. Clean whitespace
                name = name.strip() if name else ""
                title = title.strip() if title else ""
                company = company.strip() if company else ""

                # 2. Check against invalid values list
                is_title_bad = not title or title.lower() in INVALID_VALUES
                is_company_bad = not company or company.lower() in INVALID_VALUES

                # 3. Filter
                if is_title_bad or is_company_bad:
                    missing_data_rows.append(row)
                else:
                    valid_rows.append({'Name': name, 'Title': title, 'Company': company})
                    
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
        return pd.DataFrame(), pd.DataFrame()

    return pd.DataFrame(valid_rows), pd.DataFrame(missing_data_rows)

# Execute Load
print("📂 Loading and cleaning data...")
# Remove Duplicates
df = df.drop_duplicates()
df, df_missing = clean_and_load_data(INPUT_FILE)

# Save missing data
if not df_missing.empty:
    df_missing.to_csv(MISSING_DATA_FILE, index=False, header=False)
    print(f"   - Saved {len(df_missing)} rows with missing info to '{MISSING_DATA_FILE}'")

print(f"✅ Total valid rows loaded: {len(df)}")

In [ ]:
if len(df) > BATCH_SIZE:
    df = df.head(BATCH_SIZE).copy()
    print(f"✂️  Dataset successfully trimmed to first {BATCH_SIZE} rows for testing.")
else:
    print(f"ℹ️  Dataset is smaller than {BATCH_SIZE}, processing all rows.")

display(df.head())

In [ ]:
def search_web(query, max_results=3):
    """Searches DuckDuckGo and returns a summary string."""
    try:
        results = DDGS().text(query, max_results=max_results)
        if not results: return ""
        return "\n".join([f"- {r['body']}" for r in results])
    except Exception as e:
        print(f"   [Search Error] {e}")
        return ""

def ask_ollama(prompt, context=""):
    """Queries the local Llama 3.1 model."""
    full_prompt = f"""
    Context:
    {context}
    
    Instruction:
    {prompt}
    
    Output Rules:
    - Output ONLY the answer requested.
    - No conversational filler (e.g., "Here is the answer").
    """
    try:
        response = ollama.chat(model=MODEL_NAME, messages=[
            {'role': 'user', 'content': full_prompt},
        ])
        return response['message']['content'].strip()
    except Exception as e:
        return f"Error: {e}"

print("✅ AI Agents initialized.")

In [ ]:
print(f"🚀 Processing {len(df)} rows...")

# 1. Split Names
def split_name(full_name):
    parts = str(full_name).strip().split()
    first = parts[0]
    last = " ".join(parts[1:]) if len(parts) > 1 else ""
    return pd.Series([first, last])

df[['First_Name', 'Last_Name']] = df['Name'].apply(split_name)

In [ ]:
# 2. Filter Senior Executives (Local AI)
print("   - Filtering for Senior Executives...")
def is_senior(title):
    res = ask_ollama(f"Is '{title}' a Senior Executive role (C-Level, Founder)? YES/NO.")
    return "YES" in res.upper()

df['Is_Senior'] = df['Title'].apply(is_senior)
df = df[df['Is_Senior']].copy()
print(f"   - {len(df)} Executives remaining.")

In [ ]:
# 3. Domain & Employment (Web Search)
print("   - Fetching Domains & Verifying Employment...")

def enrich_data(row):
    # Get Domain
    domain_context = search_web(f"official website domain for {row['Company']}")
    domain = ask_ollama(f"Extract main domain (e.g. google.com) from: {domain_context}", domain_context)
    domain = domain.lower().replace("www.", "").split('/')[0]
    
    # Verify Employment
    work_context = search_web(f"{row['Name']} {row['Company']} 2025 LinkedIn?")
    still_working = ask_ollama(f"Is {row['Name']} working in {row['Company']} at the position of {row['Title']} currently? Answer 'Likely Yes', 'Likely No', or 'Uncertain'.", work_context)
    
    return pd.Series([domain, still_working])

if not df.empty:
    df[['Domain', 'Employment_Status']] = df.apply(enrich_data, axis=1)

In [ ]:
# 4. Generate Emails into 2 Fields
print("   - Generating Emails (splitting into 2 columns)...")

def get_emails(row):
    if row['Domain'] in ['n/a', '']: 
        return pd.Series(["N/A", "N/A"])
    
    pattern_ctx = search_web(f"email format for {row['Company']} {row['Domain']}")
    
    # Prompt specific for comma separation
    prompt = f"""
    Generate 2 likely email addresses for {row['First_Name']} {row['Last_Name']} {row['Domain']} based on context.
    Output ONLY the two emails separated by a comma (e.g. email1, email2).
    Do not add numbering or extra text.
    """
    response = ask_ollama(prompt, pattern_ctx)
    
    # Logic to split string into two fields
    try:
        # Remove newlines and split by comma
        emails = response.replace('\n', ' ').split(',')
        email1 = emails[0].strip()
        email2 = emails[1].strip() if len(emails) > 1 else ""
    except:
        email1, email2 = response, ""
        
    return pd.Series([email1, email2])

if not df.empty:
    # Assigning to two new columns directly
    df[['Email_1', 'Email_2']] = df.apply(get_emails, axis=1)

df.head()

In [ ]:
# Define final column order
final_columns = [
    'Name', 
    'First_Name', 
    'Last_Name', 
    'Title', 
    'Company', 
    'Domain', 
    'Employment_Status', 
    'Email_1', 
    'Email_2'
]

if not df.empty:
    # Ensure all columns exist before saving (handles case where filtering removed everything)
    available_cols = [c for c in final_columns if c in df.columns]
    df[available_cols].to_csv(OUTPUT_FILE, index=False)
    print(f"✅ Done! Processed data saved to: {OUTPUT_FILE}")
else:
    print("⚠️ No data remaining after filtering.")